In [1]:
import requests
import pandas as pd
import numpy as np
from plyer import notification

# Função para exibir alertas
def alerta(titulo, mensagem):
    notification.notify(
        title=titulo,
        message=mensagem,
        app_name="API Data Wrangling",
        timeout=10
    )

# URL da API Rest Countries
url = "https://restcountries.com/v3.1/all"

try:
    # Extração de dados da API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
    else:
        alerta("Erro na API", f"Código de status {response.status_code}")
        exit()

    # Transformando dados em DataFrame
    df = pd.json_normalize(data)

    # 1. Ajuste de nomes de colunas
    df.columns = [col.lower().replace(".", "_").replace(" ", "_") for col in df.columns]

    # Garantir que as colunas de interesse estejam presentes
    colunas_interesse = [
        "name_common",  # Nome do país
        "region",  # Continente
        "population",  # População
        "languages",  # Línguas
        "currencies"  # Moedas
    ]

    colunas_disponiveis = [col for col in colunas_interesse if col in df.columns]

    # Selecionar apenas as colunas disponíveis
    df = df[colunas_disponiveis]

    # Renomear colunas para nomes mais amigáveis
    df.rename(
        columns={
            "name_common": "country",
            "region": "continent",
            "population": "population",
            "languages": "languages",
            "currencies": "currencies"
        },
        inplace=True
    )

    # 2. Ajuste de valores ausentes
    df["population"] = df["population"].fillna(0)  # Substituir população ausente por 0
    df["continent"] = df["continent"].fillna("Unknown")  # Substituir continente por 'Unknown'

    # 3. Tratamento de 'languages' e 'currencies', caso existam
    if "languages" in df.columns:
        df["languages"] = df["languages"].apply(lambda x: x if isinstance(x, dict) else {})
        df_languages = df.explode("languages")
        df_languages["language"] = df_languages["languages"].apply(lambda x: list(x.values())[0] if isinstance(x, dict) else None)
    else:
        df_languages = pd.DataFrame(columns=["country", "language"])

    if "currencies" in df.columns:
        df["currencies"] = df["currencies"].apply(lambda x: x if isinstance(x, dict) else {})
        df_currencies = df.explode("currencies")
        df_currencies["currency"] = df_currencies["currencies"].apply(lambda x: list(x.keys())[0] if isinstance(x, dict) else None)
    else:
        df_currencies = pd.DataFrame(columns=["country", "currency"])

    # 4. Stack e Unstack
    df_stacked = df.set_index("country").stack()
    df_unstacked = df_stacked.unstack()

    # Exibindo os resultados
    print("Base Tratada:")
    print(df.head())

    print("\nLinguagens Explodidas:")
    print(df_languages.head())

    print("\nMoedas Explodidas:")
    print(df_currencies.head())

    print("\nDados Stack:")
    print(df_stacked.head())

    print("\nDados Unstack:")
    print(df_unstacked.head())

except Exception as e:
    alerta("Erro de Execução", f"Erro ao processar dados: {str(e)}")


Base Tratada:
         country  continent  population
0  South Georgia  Antarctic          30
1        Grenada   Americas      112519
2    Switzerland     Europe     8654622
3   Sierra Leone     Africa     7976985
4        Hungary     Europe     9749763

Linguagens Explodidas:
Empty DataFrame
Columns: [country, language]
Index: []

Moedas Explodidas:
Empty DataFrame
Columns: [country, currency]
Index: []

Dados Stack:
country                  
South Georgia  continent     Antarctic
               population           30
Grenada        continent      Americas
               population       112519
Switzerland    continent        Europe
dtype: object

Dados Unstack:
               continent population
country                            
South Georgia  Antarctic         30
Grenada         Americas     112519
Switzerland       Europe    8654622
Sierra Leone      Africa    7976985
Hungary           Europe    9749763


Criar o Virtual Environment

1.Abra o terminal na pasta do seu projeto.

2.Execute o comando abaixo para criar um ambiente virtual chamado venv: python -m venv venv

3.Ative o ambiente virtual: venv\Scripts\activate


Instalar as Dependências do Projeto

No ambiente virtual, instale as bibliotecas necessárias para o projeto:

In [3]:
pip install pandas requests plyer

Note: you may need to restart the kernel to use updated packages.


Se precisar de versões específicas:

In [4]:
pip install pandas==1.5.0 requests==2.28.0 plyer==2.0.0

Note: you may need to restart the kernel to use updated packages.Collecting pandas==1.5.0
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.9 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/10.9 MB 3.7 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/10.9 MB 4.7 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/10.9 MB 6.0 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/10.9 MB 7.2 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/10.9 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 10.9/10.9 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: plyer
    Found existing installation: plyer 2.1.0
    Uninstalling plyer-2.1.0:
      Successfully uninstalled plyer-2.1.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.0 which is incompatible.
yfinance 0.2.50 requires requests>=2.31, but you have requests 2.28.0 which is incompatible.


Salvando as versões utilizadas no arquivo requirements.txt:

In [5]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
